ReLU activation function

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [2]:
# Checkpoint function 
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path = os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print('[*] Failed to find a checkpoint')
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
# Load mnist
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    
    train_data = np.expand_dims(train_data, axis = -1)
    test_data = np.expand_dims(test_data, axis = -1)
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10)
    test_lables = to_categorical(test_labels, 10)
    
    return train_data, train_labels, test_data, test_lables
    
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

In [4]:
# Define loss
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits,
                                                                   y_true=labels,
                                                                   from_logits=True))
    return loss
    
def accuracy_fn(model, images, labels):
    logits = model(images, training = False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with  tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
#create network
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, use_bias = True, 
                                 kernel_initializer=weight_init)


def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(relu())
        
    model.add(dense(label_dim, weight_init))
    
    return model

In [7]:
# Experiments (parameters)
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
shuffle(buffer_size = 100000).prefetch(buffer_size = batch_size).\
batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
shuffle(buffer_size = 100000).prefetch(buffer_size = len(test_x)).\
batch(len(test_x))


In [8]:
# Experiments (model)
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [9]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir = logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(' [*] Load SUCCESS')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(' [!] Load failed...')
        
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network, train_input, train_label)
                    
                tf.summary.scalar(name = 'train_loss', data = train_loss, step = counter)
                tf.summary.scalar(name = 'train_accuracy', data = train_accuracy, step=counter)
                tf.summary.scalar(name = 'test_accuracy', data = test_accuracy, step=counter)
                
                print("Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,\
                       test_accuracy))
                
                counter += 1
        checkpoint.save(file_prefix = checkpoint_prefix + '-{}'.format(counter))
        
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
    print('test_Accuracy: %.4f' %(test_accuracy))

 [*] Reading checkpoints...
[*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.5409, train_loss: 2.11890984, train_accuracy: 0.3828, test_Accuracy: 0.3828
Epoch: [ 0] [    1/  468] time: 0.6458, train_loss: 2.10528731, train_accuracy: 0.3672, test_Accuracy: 0.3672
Epoch: [ 0] [    2/  468] time: 0.7487, train_loss: 2.02702618, train_accuracy: 0.3828, test_Accuracy: 0.3828
Epoch: [ 0] [    3/  468] time: 0.8857, train_loss: 1.88622355, train_accuracy: 0.5703, test_Accuracy: 0.5703
Epoch: [ 0] [    4/  468] time: 0.9846, train_loss: 1.85003161, train_accuracy: 0.6094, test_Accuracy: 0.6094
Epoch: [ 0] [    5/  468] time: 1.0756, train_loss: 1.72360229, train_accuracy: 0.7266, test_Accuracy: 0.7266
Epoch: [ 0] [    6/  468] time: 1.1685, train_loss: 1.68855858, train_accuracy: 0.6641, test_Accuracy: 0.6641
Epoch: [ 0] [    7/  468] time: 1.2945, train_loss: 1.52614379, train_accuracy: 0.6875, test_Accuracy: 0.6875
Epoch: [ 0] [    8/  468] time: 1.4025, 

Epoch: [ 0] [   75/  468] time: 8.0271, train_loss: 0.40468436, train_accuracy: 0.8672, test_Accuracy: 0.8672
Epoch: [ 0] [   76/  468] time: 8.1441, train_loss: 0.42594743, train_accuracy: 0.8594, test_Accuracy: 0.8594
Epoch: [ 0] [   77/  468] time: 8.2420, train_loss: 0.44496605, train_accuracy: 0.9062, test_Accuracy: 0.9062
Epoch: [ 0] [   78/  468] time: 8.3389, train_loss: 0.51862097, train_accuracy: 0.8359, test_Accuracy: 0.8359
Epoch: [ 0] [   79/  468] time: 8.4309, train_loss: 0.32866436, train_accuracy: 0.9062, test_Accuracy: 0.9062
Epoch: [ 0] [   80/  468] time: 8.5418, train_loss: 0.27875400, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [   81/  468] time: 8.6368, train_loss: 0.28270364, train_accuracy: 0.8984, test_Accuracy: 0.8984
Epoch: [ 0] [   82/  468] time: 8.7357, train_loss: 0.33914107, train_accuracy: 0.8906, test_Accuracy: 0.8906
Epoch: [ 0] [   83/  468] time: 8.8317, train_loss: 0.29265392, train_accuracy: 0.9062, test_Accuracy: 0.9062
Epoch: [ 0

Epoch: [ 0] [  149/  468] time: 15.3316, train_loss: 0.15314385, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  150/  468] time: 15.4456, train_loss: 0.26819849, train_accuracy: 0.9062, test_Accuracy: 0.9062
Epoch: [ 0] [  151/  468] time: 15.5366, train_loss: 0.16794384, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [  152/  468] time: 15.6275, train_loss: 0.27692825, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [  153/  468] time: 15.7416, train_loss: 0.22039965, train_accuracy: 0.9062, test_Accuracy: 0.9062
Epoch: [ 0] [  154/  468] time: 15.8355, train_loss: 0.31731099, train_accuracy: 0.8906, test_Accuracy: 0.8906
Epoch: [ 0] [  155/  468] time: 15.9305, train_loss: 0.21492718, train_accuracy: 0.9219, test_Accuracy: 0.9219
Epoch: [ 0] [  156/  468] time: 16.0234, train_loss: 0.36510977, train_accuracy: 0.8906, test_Accuracy: 0.8906
Epoch: [ 0] [  157/  468] time: 16.1364, train_loss: 0.14872438, train_accuracy: 0.9609, test_Accuracy: 0.9609
E

Epoch: [ 0] [  224/  468] time: 22.9182, train_loss: 0.11607106, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  225/  468] time: 23.0131, train_loss: 0.15780646, train_accuracy: 0.9531, test_Accuracy: 0.9531
Epoch: [ 0] [  226/  468] time: 23.1071, train_loss: 0.19812438, train_accuracy: 0.9375, test_Accuracy: 0.9375
Epoch: [ 0] [  227/  468] time: 23.2421, train_loss: 0.14715350, train_accuracy: 0.9531, test_Accuracy: 0.9531
Epoch: [ 0] [  228/  468] time: 23.3380, train_loss: 0.13520095, train_accuracy: 0.9531, test_Accuracy: 0.9531
Epoch: [ 0] [  229/  468] time: 23.4290, train_loss: 0.14288063, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [  230/  468] time: 23.5329, train_loss: 0.12507144, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  231/  468] time: 23.6238, train_loss: 0.10467823, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  232/  468] time: 23.7198, train_loss: 0.13811803, train_accuracy: 0.9609, test_Accuracy: 0.9609
E

Epoch: [ 0] [  297/  468] time: 30.1571, train_loss: 0.21221313, train_accuracy: 0.9375, test_Accuracy: 0.9375
Epoch: [ 0] [  298/  468] time: 30.2531, train_loss: 0.12532043, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  299/  468] time: 30.3430, train_loss: 0.17007473, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [  300/  468] time: 30.4750, train_loss: 0.07883897, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  301/  468] time: 30.5659, train_loss: 0.15881675, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [  302/  468] time: 30.6549, train_loss: 0.09568530, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  303/  468] time: 30.7448, train_loss: 0.13295659, train_accuracy: 0.9609, test_Accuracy: 0.9609
Epoch: [ 0] [  304/  468] time: 30.8498, train_loss: 0.21064171, train_accuracy: 0.9141, test_Accuracy: 0.9141
Epoch: [ 0] [  305/  468] time: 30.9397, train_loss: 0.08943844, train_accuracy: 0.9766, test_Accuracy: 0.9766
E

Epoch: [ 0] [  373/  468] time: 37.5987, train_loss: 0.13958400, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  374/  468] time: 37.6917, train_loss: 0.10795464, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  375/  468] time: 37.7836, train_loss: 0.19166547, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [  376/  468] time: 37.8716, train_loss: 0.08267005, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  377/  468] time: 37.9745, train_loss: 0.11230215, train_accuracy: 0.9609, test_Accuracy: 0.9609
Epoch: [ 0] [  378/  468] time: 38.0715, train_loss: 0.18483758, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [  379/  468] time: 38.1654, train_loss: 0.06714739, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  380/  468] time: 38.2574, train_loss: 0.09845620, train_accuracy: 0.9531, test_Accuracy: 0.9531
Epoch: [ 0] [  381/  468] time: 38.3664, train_loss: 0.15829808, train_accuracy: 0.9453, test_Accuracy: 0.9453
E

Epoch: [ 0] [  446/  468] time: 44.7687, train_loss: 0.18262397, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  447/  468] time: 44.8827, train_loss: 0.11277477, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  448/  468] time: 44.9767, train_loss: 0.12666506, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  449/  468] time: 45.0707, train_loss: 0.09532401, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  450/  468] time: 45.1736, train_loss: 0.17096463, train_accuracy: 0.9609, test_Accuracy: 0.9609
Epoch: [ 0] [  451/  468] time: 45.2665, train_loss: 0.20576848, train_accuracy: 0.9141, test_Accuracy: 0.9141
Epoch: [ 0] [  452/  468] time: 45.3575, train_loss: 0.08951421, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  453/  468] time: 45.4504, train_loss: 0.10028897, train_accuracy: 0.9609, test_Accuracy: 0.9609
Epoch: [ 0] [  454/  468] time: 45.5584, train_loss: 0.17026666, train_accuracy: 0.9688, test_Accuracy: 0.9688
E

와 정확도 엄청 올라감